In [ ]:
import parsing

demo_file_name = r"D:/Samihan/Documents/Programming/Python Projects/CSGO Demo Parsing/demos/cloud9-vs-outsiders-m1-dust2.dem"
print(f"Parsing {demo_file_name}")
data = parsing.parse_demo_file(demo_file_name)
# print(data.keys())

In [ ]:
# Loading demo from .json
import models
import json

with open("cloud9-vs-outsiders-m1-dust2.json", "r") as file:
# with open("cloud9-vs-outsiders-m2-ancient.json", "r") as file:
    data = json.load(file)

demo = models.deserialize_demo_data(data)

# Creating timeline and events from demo
import timelining

timeline = timelining.create_timeline(demo)
# print(len(timeline.events))
# for event in timeline.events:
#     print(event)

In [ ]:
import timelining

round_boundary_ticks: list[int] = [(round.frames[0].tick, round.frames[-1].tick) for round in demo.game_rounds]

round_frame_ticks: list[list[int]] = []
for round in demo.game_rounds:
    round_frame_ticks.append([frame.tick for frame in round.frames])

organized_events: list[timelining.Event] = [[] for _ in range(0, len(demo.game_rounds))]
for event in timeline.events:
    for frame_index, frame_ticks in enumerate(round_frame_ticks):
        if event.tick in frame_ticks:
            organized_events[frame_index].append(event)
            break
# print(len(organized_events))
# print([len(frame) for frame in organized_events])

# import plotting

# round = demo.game_rounds[0]

# plotting.plot_round(f"round_{round.round_number}", round, map_name=demo.map_name)

import plotting
round = demo.game_rounds[0]
import cProfile
import pstats
with cProfile.Profile() as pr:
    plotting.plot_round(f"round_{round.round_number}", round, map_name=demo.map_name)

stats = pstats.Stats(pr)
stats.sort_stats(pstats.SortKey.TIME)
stats.print_stats()
stats.dump_stats(filename="info-beta-2.prof")

# TODO: Organize events into frames so each frame of the GIF can have the associated events grouped with it

# TODO: Increase frame drawing speed (drawing view vectors is probably what takes the most time so maybe optimize that)

In [ ]:
# Writing events to csv

import csv
from dataclasses import fields

with open("events.csv", "w") as file:
    writer = csv.writer(file)
    all_fields: list[str] = []
    for event in timeline.events:
        all_fields.extend([field.name for field in fields(event)])
    unique_fields = list(dict.fromkeys(all_fields))
    # print(unique_fields)
    writer.writerow([field for field in unique_fields])
    for event in timeline.events:
        writer.writerow([getattr(event, field, None) for field in unique_fields])

# Writing event descriptions to .csv

import csv
from dataclasses import fields

with open("event_descriptions.csv", "w") as file:
    writer = csv.writer(file)
    writer.writerow(["tick", "name", "description"])
    for event in timeline.events:
        writer.writerow([event.tick, event.__class__.__name__, str(event)])

In [ ]:
import mathing
import numpy as np

v1 = np.array([(-2.0, -2.0), (2.0, 2.0)])
v2 = np.array([(2.0, 2.0), (-3.0, 3.0)])

print(mathing.get_line_segment_intersection_points(v1, v2))